In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import query_hive_ssh
import re
import copy
import time
import numpy as np
import os


In [2]:
def transfer_table(params, dry = False):
    hdfs_path = '/user/hive/warehouse/enwiki.db/%(table)s' % params
    stat2_path = '/home/ellery/detox/data/retention/%(name)s' % params
    local_path = '/Users/ellerywulczyn/detox/data/retention/%(name)s/' % params
    

    if not dry:
        # transfer from HDFS to stat2
        cmd = "ssh stat1002.eqiad.wmnet 'rm -rf %s'" % stat2_path
        print(os.system(cmd))
        cmd = "ssh stat1002.eqiad.wmnet 'hadoop fs -copyToLocal %s %s '" % (hdfs_path, stat2_path)
        print(os.system(cmd))
        #transfer from stat2 to local
        cmd = 'rm -rf %s' % local_path
        print(os.system(cmd))
        cmd = 'rsync -avz  stat1002.eqiad.wmnet:%s/* %s' % (stat2_path, local_path)
        os.system(cmd)
        
        
    dfs = []

    for filename in os.listdir(local_path):
        dfs.append(pd.read_csv(os.path.join(local_path,filename), sep = '\t', header = None))

    df = pd.concat(dfs)
    df.columns = params['columns']

    df = df[df['user_text'] != "false"]

    df.to_csv(local_path[:-1] + '.tsv', sep ='\t', index = False)

In [3]:
params = {
        'table':'daily_edit_counts',
        'name': 'daily_edit_counts',
        'columns': ['user_id','user_text','day','ns,', 'n']
    }
transfer_table(params, dry = False)

0
0
0


In [4]:
params = {
        'table':'user_registration',
        'name': 'user_registration',
        'columns': ['user_id','user_text', 'user_registration']
    }
transfer_table(params, dry = False)

0
0
0
